<a href="https://colab.research.google.com/github/cs145442/nlp-projects-with-tf2/blob/master/training_with_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [ ]:
!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 61kB 3.4MB/s 


In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])

  return model

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')                       

In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 384 and the optimal learning rate for the optimizer
is 0.001.



In [ ]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.4747 - accuracy: 0.8304 - val_loss: 0.4310 - val_accuracy: 0.8481
Epoch 2/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.3606 - accuracy: 0.8690 - val_loss: 0.3635 - val_accuracy: 0.8680
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3201 - accuracy: 0.8819 - val_loss: 0.3911 - val_accuracy: 0.8541
Epoch 4/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2978 - accuracy: 0.8894 - val_loss: 0.3352 - val_accuracy: 0.8786
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2802 - accuracy: 0.8972 - val_loss: 0.3427 - val_accuracy: 0.8770
Epoch 6/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2645 - accuracy: 0.9012 - val_loss: 0.3270 - val_accuracy: 0.8843
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2522 - accuracy: 0.9063 - val_loss: 0.3459 - val_accuracy: